# Traveling Salesman Problem
## Juan Marquina Cancino
## Ricardo Antonio Gutiérrez Esparza

In [1]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
import math
from functools import reduce

## 1) Read from file

In [2]:
def readSourceFile(filePath):
    r"""Recieves file path to a .csv table, representing ther distances matrix of a graph. First column and first row
    must be the nodes names, values should be separated by commas"""
    df = pd.read_csv(filePath)
    nodesNames = np.array(np.array(df.columns[1:]))
    distanceMatrix = np.array(df.values[:,1:])
    nodesDictionary = dict(zip(nodesNames, np.arange(len(nodesNames))))
    return nodesNames, nodesDictionary, distanceMatrix

In [3]:
path = '../dataset/'
fileName = 'adjacency_matrix.csv'
nodesNames, nodesDictionary, distanceMatrix = readSourceFile(path + fileName)
print("Nodes namnes:", nodesNames)
print("Nodes dictionary:", nodesDictionary)
print("Distance matrix:", distanceMatrix, sep='\n')


Nodes namnes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
Nodes dictionary: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9}
Distance matrix:
[[0 59 48 69 54 91 55 34 81 72]
 [59 0 45 19 45 73 48 13 13 42]
 [48 45 0 97 11 65 12 58 70 45]
 [69 19 97 0 31 94 23 66 71 13]
 [54 45 11 31 0 95 37 47 24 49]
 [91 73 65 94 95 0 62 35 38 61]
 [55 48 12 23 37 62 0 82 96 47]
 [34 13 58 66 47 35 82 0 10 47]
 [81 13 70 71 24 38 96 10 0 63]
 [72 42 45 13 49 61 47 47 63 0]]


## 2) Generate N Nodes

In [4]:
def NumberToLetter(N):
    numberOfLetters = ord('Z') - ord('A') + 1
    asciiA = ord('A')
    result = ''
    while(N >= 0):
        result = chr(asciiA + N%26) + result
        N = int(N/26)
        if(N == 0):
            break
    return result


In [5]:
def CalculateDistance(A, B):
    return math.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)

In [6]:
def GenerateNodes(numberOfNodes, limitLow, limitHigh):
    nodesNames = np.array([NumberToLetter(x) for x in np.arange(numberOfNodes)])
    nodesDictionary = dict(zip(nodesNames, np.arange(numberOfNodes)))
    rng = default_rng(16072001)
    nodesPosition = [ (rng.random()*(limitHigh-limitLow) + limitLow, rng.random()*(limitHigh-limitLow) + limitLow) for x in range(numberOfNodes)]
    distanceMatrix = np.array(list([CalculateDistance(nodesPosition[i], nodesPosition[j]) for j in range(numberOfNodes)] for i in range(numberOfNodes)))
    return nodesNames, nodesPosition, nodesDictionary, distanceMatrix

In [7]:
numberOfNodes = 4
limitLow = -100
limitHigh = 100
nodesNames, nodesPosition, nodesDictionary, distanceMatrix = GenerateNodes(numberOfNodes, limitLow, limitHigh)
print('Nodes Names:', nodesNames)
print('Nodes Position:', nodesPosition)
print('Nodes Dictionary: ', nodesDictionary)
print('Distance Matrix', distanceMatrix, sep='\n')

Nodes Names: ['A' 'B' 'C' 'D']
Nodes Position: [(86.74836895157574, 96.4943965054594), (-86.63483376855473, 84.04208524778474), (-95.87684156945602, 25.91066175533649), (50.77233455321419, 38.813054548947036)]
Nodes Dictionary:  {'A': 0, 'B': 1, 'C': 2, 'D': 3}
Distance Matrix
[[  0.         173.82978755 195.79078408  67.98096984]
 [173.82978755   0.          58.86150784 144.65958359]
 [195.79078408  58.86150784   0.         147.21566696]
 [ 67.98096984 144.65958359 147.21566696   0.        ]]


## 3) Objective Function 

In [8]:
def ObjectiveFunction(nodesNames, nodesDictionary, distanceMatrix, sol):
    r"""Solution is given in index form"""
    solution = list(sol)
    solution.append(solution[0])
    f = 0
    for i in range(1, len(solution)):
        f += distanceMatrix[solution[i-1]][solution[i]]
    return f


In [10]:
rng = default_rng(16072001)
solution = rng.permutation(numberOfNodes)
f = ObjectiveFunction(nodesNames, nodesDictionary, distanceMatrix, solution)
print(f)

447.8879321962664


## 4) Nodes positions randomly

In [11]:
def SolutionPermutation(nodesNames):
    numberOfNodes = len(nodesNames)
    rng = default_rng(16072001)
    return rng.permutation(numberOfNodes)

In [19]:
print(SolutionPermutation(nodesNames))

[1 0 3 2]


## 5) Genetic algorithm

In [36]:
def ParentSelectionRoulette(fitness):
    '''
        Input:  fitness -> Array of pop_size elements
        Output: Index of selected parent
    '''
    total = np.sum(fitness)
    roulette = np.cumsum(fitness / total)
    # Change to binary search
    i = 0
    dart = np.random.uniform()
    while roulette[i] < dart:
        i += 1
    return i

In [84]:
def ParentSelectionTournament(fitness, k):
    '''
        Input:  fitness -> Array of pop_size elements
                k -> size of tournament
        Output: Index of selected parent
    '''
    rng = default_rng(16072001)
    selection = rng.permutation(len(fitness))
    not_selection = selection[k:]
    raffle = fitness.copy()
    raffle[not_selection] = np.max(raffle) + 1
    print(raffle)
    return np.argmin(raffle)
    

In [38]:
fitness

array([8.77736248, 5.80120472, 3.2089305 , 8.52776972, 2.05250419,
       8.41426473, 5.39832572, 4.74720372, 5.20384386, 4.03032114,
       3.43745646, 4.93969392, 6.20628733, 3.42437813, 8.49838074,
       8.40436052, 2.05215469, 5.33148754, 4.57051908, 6.67993079])

In [72]:
selection = [4, 5, 6]

In [73]:
cpy = fitness.copy()

In [74]:
cpy[selection] = 0
cpy

array([8.77736248, 5.80120472, 3.2089305 , 8.52776972, 0.        ,
       0.        , 0.        , 4.74720372, 5.20384386, 4.03032114,
       3.43745646, 4.93969392, 6.20628733, 3.42437813, 8.49838074,
       8.40436052, 2.05215469, 5.33148754, 4.57051908, 6.67993079])

## 6) Plot the final solution